## Mathematik für Biologiestudierende II

Sommersemester 2024

02.07.2024

&copy; 2024 Prof. Dr. Rüdiger W. Braun 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.formula.api as smf
import seaborn as sns
sns.set_theme()
import warnings
warnings.filterwarnings('ignore', message='The figure layout has changed')

# Klausur

Sie können sich jetzt zur Klausur am 06.08.2024 anmelden.

Es gibt zwei verschiedene Aufgabensaätze

* Klausur mit den Themen vor dem WS 2023/24
* Klausur mit den Themen ab dem WS 2024/24

* neue Klausuraufgaben bekommt
  * wer im WS 2023/24 oder im SS 2024 Übungspunkte gesammelt hat
  * wer keine Übungspunkte hat und sich im Oktober 2023 oder später erstmals im LSF zur Vorlesung angemeldet hat
* alte Klausuraufgaben bekommt
  * wer vor dem WS 2023/24 Übungspunkte gesammelt hat
  * wer keine Übungspunkte hat und sich vor Oktober 2023 im LSF zur Vorlesung angemeldet hat
* wer sowohl vor als auch nach Oktober 2023 Übungspunkte bekommen hat und sich anmeldet, wird von mir angeschrieben

* Mit der Aufteilung auf die Hörsäle teile ich mit, welche Aufgaben jemand bekommen wird
* Dann kann man widersprechen, aber bitte zügig

#### Themen heute

* Lineare Modelle mit kategoriellen Daten
* ANOVA als lineares Modell
* Regression im exponentiellen Modell
* Halblogarithmische Darstellung
* Halbwerts- und Verdoppelungszeit

# Lineare Modelle mit kategoriellen Daten

Wir kommen zu dem Rattenbeispiel aus Lektion 21 zurück:

* kontaminiertes Gelände: fange 10 Ratten
* unbelastetes Vergleichsgelände:  fange 10 Ratten
* für jede Ratte wird ihr Alter in Monaten und der Bleigehalt im Gewebe bestimmt

In [ ]:
df = pd.read_csv('ratten.csv')

In [ ]:
sns.lmplot(df, x='Alter', y='Belastung', hue='Gelände');

* Der t-Test zeigte keinen Unterschied zwischen den Ratten auf kontaminierten und nicht kontaminiertem Gelände.
* Die Ratten auf dem kontaminierten Gelände sind aber im Schnitt jünger.  
* Wir wollen gleichaltrige Ratten vergleichen

In [ ]:
formel = 'Belastung ~ Alter + Gelände'
modell = smf.ols(formel, df)
res = modell.fit()

In [ ]:
res.summary()

* `Gelände[T.unkontaminiert]` ist signifikant
* Allerdings ist das der Unterschied bei Alter = 0
* Das ist Unsinn

* Wir vergleichen im Alter von 8 und 9 Monaten

In [ ]:
anfrage = pd.DataFrame()
anfrage['Alter'] = [8,8,9,9]
anfrage['Gelände'] = ['kontaminiert', 'unkontaminiert', 'kontaminiert', 'unkontaminiert']
anfrage

In [ ]:
res.get_prediction(anfrage).summary_frame()

* Relevant sind hier die Konfidenzintervalle für die Mittelwerte
* Für beide Werte von `Alter` ist die untere Vertrauensgrenze für die Belastung im Gewebe der Ratten vom kontaminierten Gelände höher als die obere Vertrauensgrenze für die Ratten vom unkontaminierten Gelände
* Zum Signifikanzniveau $\alpha = 0.95$ ist der Unterschied in der Bleibelastung signifikant

## Bestimmung des p-Werts

* Der p-Wert wird nur für den Unterschies beim Alter 0 ausgegeben.
* Trick:  Verlegung des Nullpunkts.

* Im Beispiel verlegen wird den Nullpunkt auf 8 Monate.
* Wir führen in der Tabelle also eine Spalte mit der Altersdifferenz zu 8 Monaten ein

In [ ]:
df['Altersdifferenz'] = df.Alter - 8
df.head()

In [ ]:
formel2 = 'Belastung ~ Altersdifferenz + Gelände'
modell2 = smf.ols(formel2, df)
res2 = modell2.fit()

In [ ]:
res2.summary()

* Der p-Wert ist 0.002

# ANOVA als lineares Modell

### Beispiel Schadstoffkonzentration

* An fünf verschiedenen Messstellen wurde die Konzentration eines Schadstoffs gemessen
* Hat die Messstelle einen Einfluss auf die Konzentration?
* Die Messstelle ist der Faktor
* Die Konzentration ist die Zielvariable

In [ ]:
u_schad = "https://www.math.uni-duesseldorf.de/~braun/bio2324/data/schadstoffe.csv"
df = pd.read_csv(u_schad, index_col=0)
df.head()

In [ ]:
g1 = df[df.Messstelle==1].Konzentration
g2 = df[df.Messstelle==2].Konzentration
g3 = df[df.Messstelle==3].Konzentration
g4 = df[df.Messstelle==4].Konzentration
g5 = df[df.Messstelle==5].Konzentration

In [ ]:
res = stats.f_oneway(g1, g2, g3, g4, g5)
res

Wir können die ANOVA auch als lineares Modell rechnen, bei dem die einzige erklärende Variable kategoriell ist

In [ ]:
formel = 'Konzentration ~ Messstelle'
modell = smf.ols(formel, df)
res = modell.fit()

In [ ]:
res.summary()

* Sehen Sie, wie das schief gegangen ist?

* kein Treatment

In [ ]:
formel = 'Konzentration ~ C(Messstelle)'
modell = smf.ols(formel, df)
res = modell.fit()

* durch `C(Messstelle)` wird festgelegt, dass es sich um einen kategoriellen Wert handelt
* wenn die Messstellen nicht durch Nummern, sondern durch Text (z.B. "unterer Bachlauf") bezeichnet sind, ist das nicht nötig

In [ ]:
res.summary()

* Der Eintrag `Prob (F-statistic)` zeigt den p-Wert der ANOVA
* Der Eintrag `F-statistic` teigt die Teststatistik der ANOVA

* Die Einträge `C(Messstelle)[T.2]` usw. zeigen **nicht** die post-Hoc Analyse
* Die muss man weiterhin wie in Lektion 17 machen

# Regression im exponentiellen Modell

## Beispiel Covid-Erkrankungen

In [ ]:
df = pd.read_csv('corona.csv')
df.head()

In [ ]:
ax = sns.scatterplot(data=df, x="Tag (im März)", y="Anzahl");

* Das Wachstum war exponentiell
* Es gab aber Schankungen durch unterschiedliche Verzögerungen der Berichte der Gesundheitsämter

# Halblogarithmische Darstellung

Bei halblogarithmischer Darstellung

* ist die $x$-Achse linear skaliert: Gleiche absolute Zuwächse pro Längeneinheit
* ist die $y$-Achse logarithmisch skaliert: Gleiche relative Zuwächse pro Längeneinheit
* Das bedeutet:  Der Logarithmus der Daten wird angezeigt, und die $y$-Achse wird entsprechend unterteilt
* Exponentiell wachsende Daten liegen bei halblogarithmischer Darstellung annäherend auf einer wachsenden Geraden, exponentiell fallende auf einer fallenden Geraden

In [ ]:
ax.set_yscale('log')
ax.figure

## Exponentielles Modell vs. Lineare Regression

* Lineares Modell: in gleichen Zeitabständen gleiche absolute Zuwächse
* Exponentielles Modell: in gleichen Zeitabständen gleiche relative Zuwächse
* Biologische Wachstums- oder Abklingprozesse verlaufen meistens exponentiell
* Aufgabe der Regression im exponentiellen Modell ist es, bei Wachstumsprozessen die Verdoppelungszeit und bei    Abklingprozessen die Halbwertszeit zu bestimmen

* Dies geschieht, indem man die Werte logarithmiert und dann deren lineare Regression berechnet

## Regression im exponentiellen Modell

* $x$ die Zeit, $z$ Daten, die exponentiell wachsen (bzw. abklingen)
* Modellgleichung für Wachstumsprozess: 
$$      z = c \cdot e^{m\cdot x}  $$
* logarithmierte Modellgleichung
$$      y = \ln(z) = \ln(c) + m \cdot x  $$

* bestimme diese Gerade durch lineare Regression
* wenn $m < 0$, dann Abklingprozess

In [ ]:
df['logAnzahl'] = np.log(df.Anzahl)
df['Tag'] = df['Tag (im März)']

In [ ]:
formel = 'logAnzahl ~ Tag'
modell = smf.ols(formel, df)
res = modell.fit()

In [ ]:
res.summary()

* m = 0.226
* b = 3.441

Die Regressionsgerade für die logarithmierten Daten ist
$$  y = 0.226 \cdot x + 3.441
$$

In [ ]:
tage = np.arange(3, 24)
gerade = 0.226*tage + 3.441

In [ ]:
titel = "Die logarithmierten Daten zusammen mit ihrer Regressionsgerade"
ax2 = sns.scatterplot(x=df.Tag, y=df.logAnzahl)
sns.lineplot(x=tage, y=gerade)
ax2.set_title(titel);

In [ ]:
titel = "Die exponentierte Regressionskurve zusammen mit den Ausgangsdaten in halblogarithmischer Darstellung"
sns.lineplot(x=tage, y=np.exp(gerade), ax=ax)
ax.set_title(titel)
ax.figure

In [ ]:
titel = "Die exponentierte Regressionskurve zusammen mit den Ausgangsdaten in linearer Darstellung"
ax.set_title(titel)
ax.set_yscale('linear')
ax.figure

# Halbwerts- bzw. Verdoppelungszeit

* Modell eines Wachstumsprozesses
$$      z = c \cdot e^{m\cdot x}  $$
* Verdoppelungszeit $t$ bestimmt durch
$$      e^{m\cdot t} = 2   $$
* Also
$$      t = \frac{\ln 2}m  $$
* Bei Abklingprozessen ist $m < 0$, dann ist
$$     t = -\frac{\ln 2}m  $$
    die Halbwertszeit

Im Beispiel Covid

In [ ]:
m = 0.226

In [ ]:
t = np.log(2) / m
t

Die Verdoppelungszeit beträgt 3.07 Tage